In [10]:
from pathlib import Path
import shutil

import cv2
from PIL import Image

from yolov5 import detect

model = 'yolov5x.pt'
file_storage = 'recognizer/file_storage'


def read_labels(labels: Path) -> list[list[str]]:
    with open(labels, 'r') as label:
        coord = label.readlines()

    boxes = []
    for box in coord:
        box = box.replace('\n', '')
        box_list = box.split()
        boxes.append(box_list)

    labels_dir = labels.parent
    shutil.copy(labels, labels_dir.parent / labels.name)
    shutil.rmtree(labels_dir, ignore_errors=True)

    return boxes


def write_image(image_path: Path, boxes: list):
    image = cv2.imread(str(image_path))
    height, width, _ = image.shape

    for box in boxes:
        x = int(float(box[1]) * width)
        y = int(float(box[2]) * height)
        cv2.circle(image, (x, y), 3, (0, 255, 0), 4)

    msg = f'Warlus number = {len(boxes)}'
    cv2.putText(image, msg, (50, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 10, cv2.LINE_AA)
    new_image = Path(image_path.parent, f'{image_path.stem}_yolov3{image_path.suffix}')
    cv2.imwrite(new_image, image)



def get_number(image_path: Path) -> int:
    im = Image.open(str(image_path))
    detect.run(
        weights=model,
        source=str(image_path),
        imgsz=im.size,
        nosave=False,
        save_txt=True,
        save_conf=True,
        project=file_storage,
    )
    labels = Path(file_storage, 'exp', f'{image_path.stem}.txt')
    boxes = read_labels(labels)
    write_image(image_path, boxes)
    #return len(boxes)

In [15]:
foto = Path('foto/787.png')
get_number(foto)

YOLOv5 🚀 v6.2-128-ge8a9c5a Python-3.10.0 torch-1.12.1+cu102 CPU

Fusing layers... 
Model summary: 206 layers, 3087256 parameters, 0 gradients, 4.2 GFLOPs
image 1/1 /home/zorkin/Yandex.Disk/Обучение/Hackathon/vladivostok_chempionat/model_train/foto/787.png: 1024x1280 4 veinss, 129.5ms
Speed: 2.1ms pre-process, 129.5ms inference, 0.5ms NMS per image at shape (1, 3, 1664, 1280)
Results saved to recognizer/file_storage/exp2
1 labels saved to recognizer/file_storage/exp2/labels


FileNotFoundError: [Errno 2] No such file or directory: 'recognizer/file_storage/exp/787.txt'